In [1]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
def read_video():

    cap = cv2.VideoCapture('ball.mov')

    frames = []
    xs = []
    ys = []

    # this reads the video, captures frames, collects 
    # and plots the coordinates of the center of the ball 
    while cap.isOpened():
        ret, frame = cap.read()

        # if frame is read correctly ret is True
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        
        # find the maks of the ball
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, (0, 130, 0), (8, 255, 255))
        mask[400:, :200] = 0
        mask[500:, :900] = 0
        kernel = np.ones((5,5),np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        # find the center location of the ball
        y_coords, x_coords = np.nonzero(mask)
        
        if len(y_coords) != 0:
            avg_x = int(np.mean(x_coords))
            avg_y = int(np.mean(y_coords))
            
            xs.append(avg_x)
            ys.append(avg_y)
            
            frame = cv2.circle(frame, (avg_x, avg_y), 5, (0, 255, 0), -1)
            
        frames.append(frame)

        cv2.imshow('frame', frame)
        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


    return frames, xs, ys


frames, xs, ys = read_video()

Can't receive frame (stream end?). Exiting ...


In [3]:
def find_curve(xs, ys):
    # find the coefficients for the parabola function

    Y = np.array([ys]).reshape(-1, 1)
    xs = np.array([xs]).reshape(-1, )
    X = np.ones(shape = (Y.shape[0], 3))
    X[:, 0] = xs*xs
    X[:, 1] = xs

    B = np.linalg.inv(X.T@X)@(X.T@Y)

    a = B[0][0]
    b = B[1][0]
    c = B[2][0]
    
        
    x_range = []
    y_domain = []

    for x in range(frames[0].shape[1]):
        y = a*x*x + b*x + c
        x_range.append(x)
        y_domain.append(y)
        
    return np.array(list(zip(x_range, y_domain)), dtype = np.int32).reshape((-1, 1, 2))

points = find_curve(xs, ys)
points.shape

(1218, 1, 2)

In [6]:
def write_video(frames, points):

    size = (frames[0].shape[1], frames[0].shape[0]) 

    # Below VideoWriter object will create 
    # a frame of above defined The output  
    # is stored in 'filename.avi' file. 
    result = cv2.VideoWriter('proj1_part1.avi',  
                            cv2.VideoWriter_fourcc(*'MJPG'), 
                            60, size) 
    
    for frame in frames: 
        frame = cv2.polylines(frame, [points], False, (0,0,255), 1)
        result.write(frame) 
        
    result.release()
    

In [7]:
write_video(frames, points)